In [13]:
%load_ext autoreload
%autoreload 2

# PROJET RI TESTS

In [1]:
import datasets
import utils
import features

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk


C:\Users\NeilB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NeilB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NeilB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## DATA

In [9]:
#T elechargement et sauvegarde des datasets 

# from datasets import load_dataset 
# wikiqa = load_dataset("wiki_qa")
# wikiqa.save_to_disk("wikiqa")

In [2]:
#Load le dataset ( enregistré en format pyarrow ./wikiqa )
wikiqa_data = datasets.load_from_disk("wikiqa")

In [3]:
print(wikiqa_data)
print(wikiqa_data["train"])

DatasetDict({
    test: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 6165
    })
    validation: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 2733
    })
    train: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 20360
    })
})
Dataset({
    features: ['question_id', 'question', 'document_title', 'answer', 'label'],
    num_rows: 20360
})


In [4]:
wikiqa_data_train = wikiqa_data["train"]    
questions = wikiqa_data_train["question"]   #questions
answers = wikiqa_data_train["answer"]       #answers
labels = wikiqa_data_train["label"]         #labels (pertinent non pertinent)

#Preprocessing ( see utils.preprocess )
questions = [utils.preprocess(txt) for txt in questions]
answers = [utils.preprocess(txt) for txt in answers]

### Overlap fraction between a query and a set of answers

In [5]:
# test de la feature overlap
qrep = ["apples banana"]
rep = ["I like apples and bananas.", "I prefer oranges."]

# Calculate overlap fraction feature for the sample data using the adapted function
overlap_fraction_features = list(features.calculate_overlap_fraction(qrep, rep))

print(overlap_fraction_features)

ValueError: too many values to unpack (expected 2)

### Overlap synonym fraction between a query and a set of answers

In [7]:
# Sample data
qrep = "sad"
rep = ["I am feeling good today.", "She seems pitiful sad and distressing and sorry."]

# Calculate overlap synonym fraction feature for the sample data using the function
overlap_syn_fraction_features = list(features.overlap_syn_fraction(qrep, rep))

print(overlap_syn_fraction_features)

print(utils.get_synonyms('sad'))

NameError: name 'tokenize' is not defined

### BM25 scores 

In [21]:
import math
from collections import Counter

query = "query bout bananas"
document = "documentary about bananas"
corpus = [
    "this is the first document with strawberries",
    "this document is the second document without apples",
    "and this is the third one",
    "is this the first document but ananas are cool",
    "documentary about bananas"
]

score = bm25_score(query, document, corpus)
print(f"BM25 score: {score}")


BM25 score: 1.8372575870263608


In [8]:
import nltk
from nltk.util import ngrams
from nltk.lm import Laplace
from nltk.lm.preprocessing import pad_both_ends

nltk.download('gutenberg')
from nltk.corpus import gutenberg

# Train a unigram language model using Gutenberg corpus
text = gutenberg.raw('shakespeare-hamlet.txt')
tokens = text.split()
train_data = [list(ngrams(pad_both_ends(tokens, n=1), n=1)) for n in range(1, 2)]
lm = Laplace(1)  # Laplace smoothing with n=1
lm.fit(train_data)

def calculate_lm_score(text):
    test_data = list(ngrams(pad_both_ends(text.split(), n=1), 1))
    score = lm.perplexity(test_data)
    return score

# Sample data
text1 = "To be or not to be"
text2 = "To sleep, perchance to dream"

# Calculate LM score for the sample data
lm_score1 = calculate_lm_score(text1)
lm_score2 = calculate_lm_score(text2)

print("LM score for text1:", lm_score1)
print("LM score for text2:", lm_score2)

[nltk_data] Downloading package gutenberg to /home/neyroy/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


ValueError: Cannot fit without a vocabulary or text to create it from.